# Question 1

Consider the RSA-OAEP implementation given the file “RSA_OAEP.py”, in which the random number R is an 8-bit unsigned integer. Using the public key

	(e, N) = (65537, 70212284026476551287497867344660173062242619935306997607985987428352052911293)

I encrypted my 4 decimal digit PIN and the resulting ciphertext is 

	c = 60400943706823506830284280114139818288715016023417103465230780522075862090739

What is my PIN? Submit your code used in finding my PIN. 



In [2]:
!pip install pyprimes
!pip install pycryptodome

  Created wheel for pyprimes: filename=pyprimes-0.1-cp36-none-any.whl size=13531 sha256=e5c5a9202ad745fb5636ebf5909f48952b2e2dbe27d0de83f9ea6628d136604c
  Stored in directory: /root/.cache/pip/wheels/be/e8/78/469e93f10ce9be93e3cd49f84bd1c76ecb42962f4f6b49bc70
Successfully built pyprimes
     |████████████████████████████████| 9.7MB 9.8MB/s 


In [0]:
import math
import timeit
import random
import pyprimes
import warnings
from Crypto.Hash import SHA3_256
from Crypto.Hash import SHA3_384
from Crypto.Hash import SHA3_512
from Crypto.Hash import SHAKE128, SHAKE256


In [4]:
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcd = b
    return gcd, x, y

def modinv(a, m):
    gcd, x, y = egcd(a, m)
    if gcd != 1:
        return None  # modular inverse does not exist
    else:
        return x % m

def random_prime(bitsize):
    warnings.simplefilter('ignore')
    chck = False
    while chck == False:
        p = random.randrange(2**(bitsize-1), 2**(bitsize)-1)
        chck = pyprimes.isprime(p)
    warnings.simplefilter('default')    
    return p
 
k0 = 8
k1 = 128

def RSA_KeyGen(bitsize):
    p = random_prime(bitsize)
    q = random_prime(bitsize)
    N = p*q
    phi_N = (p-1)*(q-1)
    e = 2**16+1
    while True:
        gcd, x, y = egcd(e, phi_N)
        if gcd == 1:
            break
        e = e+2
    d = modinv(e, phi_N)    
    return e, d, p, q, N

def RSA_OAEP_Enc(m, e, N, R):
    k = N.bit_length()-2
    m0k1 = m << k1
    shake = SHAKE128.new(R.to_bytes(k0//8, byteorder='big'))
    GR =  shake.read((k-k0)//8)
    m0k1GR = m0k1 ^ int.from_bytes(GR, byteorder='big')
    shake = SHAKE128.new(m0k1GR.to_bytes((m0k1GR.bit_length()+7)//8, byteorder='big'))
    Hm0k1GR =  shake.read(k0//8)
    RHm0k1GR = R ^ int.from_bytes(Hm0k1GR, byteorder='big')
    m_ = (m0k1GR << k0) + RHm0k1GR
    c = pow(m_, e, N)
    return c

def RSA_OAEP_Dec(c, d, N):
    k = N.bit_length()-2
    m_ = pow(c, d, N)
    m0k1GR = m_ >> k0
    RHm0k1GR =  m_ % 2**k0
    shake = SHAKE128.new(m0k1GR.to_bytes((m0k1GR.bit_length()+7)//8, byteorder='big'))
    Hm0k1GR =  shake.read(k0//8)
    R = int.from_bytes(Hm0k1GR, byteorder='big') ^ RHm0k1GR
    shake = SHAKE128.new(R.to_bytes(k0//8, byteorder='big'))
    GR =  shake.read((k-k0)//8)
    m0k1 = m0k1GR ^ int.from_bytes(GR, byteorder='big')
    m = m0k1 >> k1
    return m

e, d, p, q, N = RSA_KeyGen(128)
print("p =", p)
print("q =", q)
print("N =", N)
print("e =", e)
print("d =", d)

print("\nTesting Encryption and Decryption Functions")
failure = 0
for i in range(0, 10):
    m = random.randint(0,10000)
    R =  random.randint(2**(k0-1), 2**k0-1)
    c = RSA_OAEP_Enc(m, e, N, R)
    if m != RSA_OAEP_Dec(c, d, N):
        failure += 1 
        
if failure != 0:
    print("Failure:(", failure)
else:
    print("Success:)")

p = 332182099476571620307858753995923821859
q = 318952866312558023359606206136855625851
N = 105950432765775798559296656910865334198357784830271880872556058697433279277009
e = 65537
d = 32543177313198144941004954508380291703920455037185678048087757937224149130473

Testing Encryption and Decryption Functions
Success:)


In [5]:
e = 65537
N = 70212284026476551287497867344660173062242619935306997607985987428352052911293
c = 60400943706823506830284280114139818288715016023417103465230780522075862090739
found = False
for R in range(2**(8-1), 2**8 -1):
  for m in range(0, 9999 + 1):
    c_prime = RSA_OAEP_Enc(m, e, N, R)
    if c_prime == c:
      print('PIN =', m)
      print('R =', R)
      found = True
      break
  if found:
    break




PIN = 5377
R = 157


PIN is 5377

R is 157

# Question 2


Consider the ElGamal encryption algorithm implemented in the file “ElGamal.py”, which contains flaw.  I used this implementation to encrypt a message using the following parameters:

	q = 1331165794223730998214479682055290809139803703979
	p = 157985265365233926063394088702502775477411699807450440916775405947257964574813502993749815770128973338289285516154109088043476314331444397215358170585840641049172791477662283893716386808139204949694492602287891654767148522867881937046157301612266431912023462991540765986938468014946969764702086638496649455657
	g = 135065956040029542891335614580458248416002250295204395146754036299690682917789289716583464736425816867965184913947179997468650756414729019331183463002574881956749358833871584578559474520218159551812002168419391427229522879948629379275361929622066470148375436287416532348407065836711249965189758444892419490190
	public key (h) =  65369531380434811091013169285144074654274264126019340876116721977646567453108441439476580614131141018711217039183159447339703498645723099709331310035001607050335740436825222938421053125935986024030566120585714682225125302549720107383365077208839310065478286374555114097276440333388769523316671044117487454589

And the resulting ciphertext is

	r = 3603216964442507357032842714265491356140106170126012271249273782498781062854993590551963255079610858746338241608699542316440867356686210833642816015952448905408390917797105408900214398591806869245572453652902222971116293353284737156497321871750301615013009395209713928847567903247743033867199791859981117263 
	t= 42244680577489180150438247901105682607063917920969521526593784819134087337617171624434658483262821880559452723151685731248400300205495303414447593079691461437018769475479437685274446474785220966939670711206064076236087750024913718835564780638938635976122622009741305316868203434730711663863398065249173925645

Can you find my message?


In [0]:
import random
import pyprimes
import warnings

In [7]:
def large_DL_Prime(q, bitsize):
    warnings.simplefilter('ignore')
    chck = False
    while chck == False:
        k = random.randrange(2**(bitsize-1), 2**bitsize-1)
        p = k*q+1
        chck = pyprimes.isprime(p)
    warnings.simplefilter('default')    
    return p

def Param_Generator(qsize, psize):
    q = random_prime(qsize)
    p = large_DL_Prime(q, psize-qsize)
    tmp = (p-1)//q
    g = 1
    while g == 1:
        alpha = random.randrange(1, p)
        g = pow(alpha, tmp, p)
    return q, p, g

# Generating private-public key pair
def Key_Gen(q, p, g):
    s = random.randint(1, q) # private key
    h = pow(g, s, p)         # public key
    return s, h

# Encryption
def Enc(m, h, q, p, g): # m is the message
    m = int.from_bytes(message, byteorder='big')
    k = random.randint(1, 2**16-1)
    r = pow(g, k, p)
    t = (pow(h, k, p)*m)%p
    return r, t

# Decryption
def Dec(r, t, s, q, p, g):
    m = (pow(r, q-s, p)*t)%p
    return m.to_bytes((m.bit_length()+7)//8, byteorder='big')

# Test
print("Testing the ElGamal Encryption and Decryption")
# Generate domain parameters (q, p, g)
q, p, g = Param_Generator(160, 1024)
print("q =", q)
print("p =", p)
print("g =", g)

# Generate private-public key pairs for a user
s, h = Key_Gen(q, p, g)
print("secret key (s):", s)
print("public key (h):", h)

# Encrypt a random message
message = b'Hello World!'
r, t = Enc(message, h, q, p, g)
print("ciphertext (r, t):", r, t)

# Decrypt the ciphertext
print("\nDecrpyted message:", Dec(r, t, s, q, p, g))

Testing the ElGamal Encryption and Decryption
q = 748996131086003887997329615486447263958817625353
p = 48546109977026151450398613760871582566229994236868692487732628689166244197073316639945675338086252973925767810369792337992208952702973319920974801746291639124850041012780701524450070197193384942586573771892688884082622515875810971233570211478206854008201046476605612523832502167081152268294903606843149001937
g = 20083891411479595924485741950434161722659720563509785763906703197012273488487263209613537557493471826826852181609803239168526818819924811114930038072656331544586488514592285609050489323588495024067354790774992210756925727064341925077396439003155035796222263708475032538483344413724714488464051909338622624871
secret key (s): 645685936722805149866501818537247685597607607166
public key (h): 15061454119313234166262493625952568253990075222508223907373819357785312171884057962941912353453101176509990994688588665731754081406425591487549394756231482509697444678961816959106208161490597359

In [8]:
# public parameters
q = 1331165794223730998214479682055290809139803703979
p = 157985265365233926063394088702502775477411699807450440916775405947257964574813502993749815770128973338289285516154109088043476314331444397215358170585840641049172791477662283893716386808139204949694492602287891654767148522867881937046157301612266431912023462991540765986938468014946969764702086638496649455657
# primitive root (Alpha)
g = 135065956040029542891335614580458248416002250295204395146754036299690682917789289716583464736425816867965184913947179997468650756414729019331183463002574881956749358833871584578559474520218159551812002168419391427229522879948629379275361929622066470148375436287416532348407065836711249965189758444892419490190
# public key (Beta)
h =  65369531380434811091013169285144074654274264126019340876116721977646567453108441439476580614131141018711217039183159447339703498645723099709331310035001607050335740436825222938421053125935986024030566120585714682225125302549720107383365077208839310065478286374555114097276440333388769523316671044117487454589
# ciphertext
r = 3603216964442507357032842714265491356140106170126012271249273782498781062854993590551963255079610858746338241608699542316440867356686210833642816015952448905408390917797105408900214398591806869245572453652902222971116293353284737156497321871750301615013009395209713928847567903247743033867199791859981117263 
t = 42244680577489180150438247901105682607063917920969521526593784819134087337617171624434658483262821880559452723151685731248400300205495303414447593079691461437018769475479437685274446474785220966939670711206064076236087750024913718835564780638938635976122622009741305316868203434730711663863398065249173925645

# compute m = t * h^-k (mod p)

# to find k we test for which k value g^k = r (mod p)
for k in range(1, 2**16-1):
  if pow(g, k, p) == r:
    m = (pow(modinv(h, p), k, p) * t) % p
    print(m.to_bytes((m.bit_length()+7)//8, byteorder='big'))
    break


b'My favorite machine at the gym is the vending machine.'


Message: 'My favorite machine at the gym is the vending machine.'

# Question 3



In Kerberos, the Ticket Granting Server includes the identity of the server (S) in its response to client: E_(K_CG ) (S,K_CS ). What would happen if the identity of the server were not included? Does it lead to an attack?

If the identity of the server is not included when sending the ticket to the client, it would lead to malicious activities. An eavesdropper could intercept the ticket sent from the ticket-granting server. Hence a meet-in-the-middle attack can occur since the server can't identify the client responsible for the request.

# Question 4


Alice and Bob wants to establish a secure channel to communicate securely. Suppose Alice and Bob have long term RSA public and private key pairs: (〖pk〗_A,〖sk〗_A) and (〖pk〗_B,〖sk〗_B), respectively. They can use both the RSA signature and encryption algorithms and they know each other’s long term public keys. Show how Alice and Bob can achieve forward secrecy.

To achieve forward secrecy, the compromise of their long term RSA keys should not compromise the past session keys. This can be achieved by using a station-to-station protocol, where Bob sends the encrypted signature (E_k(sign_b(R_B, R_A))) to Alice, this doesn't allow the attacker to compromise a previous session key if they obtain Bob's private key b.

# Question 5

Consider the DSA scheme implemented in the file “DSA.py”. The public parameters and public key are:

	q = 897434149680309024926610536586679400252190817513
	p = 97223004199266313523049166053330029092380541300786138924873181088471438705453794046370914345592432368059271294544102722787957310837797304650943069820520287549826630230617625792526214799206486444554607275157031742808122667064876655138748567945051878459968434840972135354745893868660267009794876094057307360271
	g = 4621497210057935612371988511711932510361318115609980978853236984314561739819039313271820105098638480214293876477070872723831493769268422441714876014396954567136665583461293138792502100498181714605761615088670098808016625617309860858682957197265294737395362167975930097648958972424479880194787709852371142579
	public key (beta): 45720223092558820344769930028614803638859051907129501277880999062740852114889610377894039520973053847174144955552627174266061939323577184681728281156812736603122999262209953001238229439108117677423857541271841004309469066208083385254271589636542160767902921803860270699359911081346969522186114311390226677995

You are given two signatures for two different message as follows:

	(message1, r_1, s_1) = (b"He who laugh last didn't get the joke", 867552604169477346883674422144796797059303863627, 243861349833858115605937030382497401412336608822)
	(message2, r_2, s_2) = (b"Ask me no questions, and I'll tell you no lies" 686145019080375810998084468514665120375929537329, 774583422188330317252601038183072854135396118762)

Also, you discovered that k_2= 2k_1  mod q. Show how you can find the secret key a. 


In [9]:
import random
import pyprimes
import warnings
from Crypto.Hash import SHA3_256
from Crypto.Hash import SHAKE128

def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcd = b
    return gcd, x, y

def modinv(a, m):
    if a < 0:
        a = a+m
    gcd, x, y = egcd(a, m)
    if gcd != 1:
        return None  # modular inverse does not exist
    else:
        return x % m
    
def random_prime(bitsize):
    warnings.simplefilter('ignore')
    chck = False
    while chck == False:
        p = random.randrange(2**(bitsize-1), 2**bitsize-1)
        chck = pyprimes.isprime(p)
    warnings.simplefilter('default')    
    return p

def large_DL_Prime(q, bitsize):
    warnings.simplefilter('ignore')
    chck = False
    while chck == False:
        k = random.randrange(2**(bitsize-1), 2**bitsize-1)
        p = k*q+1
        chck = pyprimes.isprime(p)
    warnings.simplefilter('default')    
    return p

def Param_Generator(qsize, psize):
    q = random_prime(qsize)
    p = large_DL_Prime(q, psize-qsize)
    tmp = (p-1)//q
    g = 1
    while g == 1:
        alpha = random.randrange(1, p)
        g = pow(alpha, tmp, p)
    return q, p, g

# Generating private-public key pair
def Key_Gen(q, p, g):
    s = random.randint(1, q) # private key
    h = pow(g, s, p)         # public key
    return s, h

# Signature generation
def Sig_Gen(message, a, k, q, p, g):
    shake = SHAKE128.new(message)
    h = int.from_bytes(shake.read(q.bit_length()//8), byteorder='big')
    r = pow(g, k, p)%q
    s = (modinv(k, q)*(h+a*r))%q
    return r, s

# Signature verification
def Sig_Ver(message, r, s, beta, q, p, g):
    shake = SHAKE128.new(message)
    h = int.from_bytes(shake.read(q.bit_length()//8), byteorder='big')
    u1 = (modinv(s, q)*h)%q
    u2 = (modinv(s, q)*r)%q
    v1 = (pow(g, u1, p)*pow(beta, u2, p)%p)%q

    if v1 == r:
        return True
    else:
        return False

# Test
print("Testing the DSA signature generation and verification")
# Generate domain parameters (q, p, g)
q, p, g = Param_Generator(160, 1024)
print("q =", q)
print("p =", p)
print("g =", g)

# Generate private-public key pairs for a user
a, beta = Key_Gen(q, p, g)
print("secret key (a):", a)
print("public key (beta):", beta)

message = b'Hello World!'
k = random.randint(0, q-1)
r, s = Sig_Gen(message, a, k, q, p, g)

if Sig_Ver(message, r, s, beta, q, p, g):
    print("signature verifies:) ")
else:
    print("invalid signature:( ")    




Testing the DSA signature generation and verification
q = 1407131678887102197884949280386553898015762812849
p = 171297130419273585820398213861753332416809888524744370580608243498491329404903991702417675890204997221189793847847186889923698561540208324467335888522414466002405214970828732676653004810078406529965117972798170184463852720401137350068216866196126824414468333907909730580641086579429545657376789490720020112527
g = 114491618344454896130764668002133322499703422962395311438288558762818875131817414595238244249276801465326228309757760583247524416947560657132503749994592834955138361725256094296991368230817603026350085411080623457030571840675707641516973029129086954241783554667557647654768512880602569834463021715961316049383
secret key (a): 164297893015201654837688004916190902757118818515
public key (beta): 367067705424975201766281631774153111971822865608574786610564947120461904549398189788076757992087117053426599809825016696442846404798460075030079471647289261740744873377098258975335

In [10]:
q = 897434149680309024926610536586679400252190817513
p = 97223004199266313523049166053330029092380541300786138924873181088471438705453794046370914345592432368059271294544102722787957310837797304650943069820520287549826630230617625792526214799206486444554607275157031742808122667064876655138748567945051878459968434840972135354745893868660267009794876094057307360271
g = 4621497210057935612371988511711932510361318115609980978853236984314561739819039313271820105098638480214293876477070872723831493769268422441714876014396954567136665583461293138792502100498181714605761615088670098808016625617309860858682957197265294737395362167975930097648958972424479880194787709852371142579
beta = 45720223092558820344769930028614803638859051907129501277880999062740852114889610377894039520973053847174144955552627174266061939323577184681728281156812736603122999262209953001238229439108117677423857541271841004309469066208083385254271589636542160767902921803860270699359911081346969522186114311390226677995

message1, r_1, s_1 = b"He who laugh last didn't get the joke", 867552604169477346883674422144796797059303863627, 243861349833858115605937030382497401412336608822
message2, r_2, s_2 = b"Ask me no questions, and I'll tell you no lies", 686145019080375810998084468514665120375929537329, 774583422188330317252601038183072854135396118762
# k2 = 2*k1 mod q
shake = SHAKE128.new(message1)
h_1 = int.from_bytes(shake.read(q.bit_length()//8), byteorder='big')
shake = SHAKE128.new(message2)
h_2 = int.from_bytes(shake.read(q.bit_length()//8), byteorder='big')
# a = (s_1*h_2 − s_2*h_1*x)*(s_2*r_1*x − s_1*r_2 )^−1(mod q)

a = ((s_1 * h_2 - s_2 * h_1 * 2)*modinv(s_2*r_1*2 - s_1*r_2,q))%q
print('a =', a)



a = 253269165174290268846821928601654697793514680881


a = 253269165174290268846821928601654697793514680881